In [26]:
# Things to change when running this on different computers are marked with *UPDATE*

from datetime import datetime
from selenium import webdriver #installation instructions: https://tinyurl.com/qec3qd8 & https://tinyurl.com/y8uqp8r4
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import pandas as pd
import numpy as np
import os
import time

#set working directory
os.chdir("C:/Users/Linfei Li/Downloads") # *UPDATE* change to your downloads 

####################
#     FUNCTIONS 
####################

# Log in as Admin to CDR SONA
def login():
    browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_userid"]').send_keys("drlra") # *UPDATE*
    browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_pw"]').send_keys("drlra") # *UPDATE*
    browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_default_auth_button"]').click()

# Navigates to Inactive Studies Page
def gotoInactive():
    browser.find_element_by_xpath('//*[@id="NavStudiesMenu"]').click()
    browser.find_element_by_xpath('//*[@id="primary-content"]/section/div[1]/div[3]/div/nav/div[2]/ul[1]/li[1]/ul/li[1]/a').click()
    browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_lnkFilterStatusInactive"]').click() 
    time.sleep(2) #wait 2 sec for page to fully load

# Navigates from Inactive to specific study [name]
def enterstudy(name):
    browser.find_element_by_link_text(name).click()

# Copies various values of interest from study information page
# returns NA if no such information exists
def getVal(xpath):
    try:
        elem=browser.find_element_by_xpath(xpath)
        val=elem.text
    except NoSuchElementException:
        return 'NA'
    return val

# returns Date Object for study
def getDate():
    try: 
        elem=browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_lblCreationDate"]')
        date=elem.text
    except NoSuchElementException:
        browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_ctl00_lblStudyMenu"]').click()
        browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_ctl00_lblAdminDownloadParticipant"]').click()
        browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_Submit_Send"]').click()
        date=browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_DataGridRosterDownload"]/tbody/tr[1]/td[5]').text
        browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_Unnamed1_lblStudyMenu"]').click()
        browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_Unnamed1_lblStudyInfo"]').click()
        return datetime.strptime(date,"%B %d, %Y %H:%M %p")
    return datetime.strptime(date,"%B %d, %Y")

# returns list of RAs
def getRA():
    lst=[]
    for i in range(4): #assumes max 4 researchers
        idtext='ctl00_ContentPlaceHolder1_repResearcher_ctl0'+str(i)+'_HyperLinkResearcher'
        try:
            lst.append(browser.find_element_by_id(idtext).text)
        except NoSuchElementException:
            break
    if not lst:
        return 'NA'
    else:
        return ','.join(lst)

# Appends "[ARCHIVED]" to study title
# for post 2015 studies
def archive():
    browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_ctl00_lblStudyMenu"]').click()
    browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_ctl00_lblAdminchangeStudyInfo"]').click()
    browser.find_element_by_xpath('//*[@id="txtExpName"]').clear()
    newtitle="[ARCHIVED] "+name
    browser.find_element_by_xpath('//*[@id="txtExpName"]').send_keys(newtitle)
    browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_Submit_Update"]').click()

# Downloads CSV of study participants
def downloadCSV():
    browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_ctl00_lblStudyMenu"]').click()
    browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_ctl00_lblAdminDownloadParticipant"]').click()
    browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_ddlListOutput"]/option[2]').click()
    browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_Submit_Send"]').click()
    
# Renames most recently downloaded participants csv file for easy reference
# taken from https://stackoverflow.com/questions/34548041/selenium-give-file-name-when-downloading
def rename_file(newname, folder_of_download):
    filename = max([f for f in os.listdir(folder_of_download)], key=lambda xa : os.path.getctime(os.path.join(folder_of_download,xa)))
    if '.part' in filename:
        time.sleep(1)
        os.rename(os.path.join(folder_of_download, filename), os.path.join(folder_of_download, newname))
    else:
        os.rename(os.path.join(folder_of_download, filename),os.path.join(folder_of_download,newname))

# Deletes a study by name
def delete_study(name):
    gotoInactive()
    enterstudy(name)
    browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_ctl00_lblStudyMenu"]').click()
    browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_ctl00_lnkAdminDeleteStudy"]').click()
    browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_btnDeleteYes"]').click()    
    
# Takes in Study Name and 
#  a) for pre-2015: outputs in working directory (Downloads)
#     1) 2-sheet Excel file (study information (filled), participants (blank)) 
#     2) csv file named "PARTICIPANTS YYYY-MM-DD [Study Name].csv"
#      It is still necessary to reformat the study information according to archive guidelines
#      and also copy past the participants csv into the blank excel sheet
#  b) for 2015 and after: updates study title on SONA
def process_study(name):
    gotoInactive()
    enterstudy(name)
    date=getDate()
    if date.year>=2015:
        archive()
        return None
    else: 
        #recording variables of interest
        name=browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_lblStudyName"]').text
        leng=browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_lblDuration"]').text
        pay=browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_lblCreditTotal"]').text
        abstr=getVal('//*[@id="ctl00_ContentPlaceHolder1_lblShortDesc"]')
        descrip=getVal('//*[@id="ctl00_ContentPlaceHolder1_lblLongDesc"]')
        prescr=getVal('//*[@id="ctl00_ContentPlaceHolder1_lblPrescreenRestriction"]')
        course=getVal('//*[@id="primary-content"]/section/div[2]/div[3]/section[1]/div/table/tbody/tr[2]/td[2]/ul/li')
        irb=getVal('//*[@id="ctl00_ContentPlaceHolder1_lblIRBApprvalExpDate"]')
        res=getRA()

        #creating Study Information
        row_names=['Study Name', 'Length', 'Pay','Abstract',
                   'Description','Prescreen restrictions','Course Restrictions',
                   'IRB protocol number','Date created','Researcher']
        d=pd.DataFrame(np.array([name,leng,pay,abstr,descrip,prescr,course,irb,date.strftime("%m/%d/%Y"),res]),index=row_names)
        infotitle=date.strftime("%Y-%m-%d")+" "+name.replace(": ","-")+".xlsx"
    
        #downloading Participants
        downloadCSV()
        parttitle="PARTICIPANTS "+date.strftime("%Y-%m-%d")+" "+name.replace(": ","-")+".csv"
        rename_file(parttitle,os.path.abspath("C:/Users/Linfei Li/Downloads") ) # *UPDATE* change to your downloads
        #participants=pd.read_csv(parttitle,encoding = 'cp1252') 
            #however, file appears to be utf-8 encoded according to different options: https://tinyurl.com/ycjlrbzk
        csv=pd.DataFrame() 
        
        #creating final Excel File
        with pd.ExcelWriter(infotitle) as writer:  
            d.to_excel(writer, sheet_name='Information')
            #participants.to_excel(writer, sheet_name='Participants')
            csv.to_excel(writer, sheet_name='Participants')
        return name

# Final function for processing list of study names
# returns list of studies to be deleted; also prints studies to be deleted to the screen
def processor(names):
    if not names:
        raise ValueError('list of study names to be processed is empty')
    
    print("Delete the following studies after checking and combining the files")
    lst=[] #list of pre-2015 studies, to be deleted
    for i in range(len(names)):
        lst.append(process_study(names[i]))
    print(lst)
    return list(filter(None, lst)) 

# Final function for deleting list of studies
# exits window at the end
def deleter(names):
    if not names:
        raise ValueError('list of study names to be deleted is empty')
        
    for i in range(len(names)):
        delete_study(names[i])
    browser.quit()

In [27]:
# list of all studies to be archived
# add more in the format of ['DRL: XXX','CDRL: XXX','CDRVL: XXX'], etc.
names=['CRL: Diet and Visual Acuity'] # *UPDATE*

In [28]:
# Open browser and SONA page
options = Options()
options.add_argument("start-maximized")
options.add_argument('--no-sandbox') #for chrome not reachable errors
browser = webdriver.Chrome(chrome_options=options)
browser.get('https://chicagocdr.sona-systems.com/')
time.sleep(2) #wait 2 sec for page to fully load

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [29]:
login()

In [30]:
to_delete=processor(names)

Delete the following studies after checking and combining the files


ParserError: Error tokenizing data. C error: Expected 1 fields in line 5, saw 2


In [ ]:
#this is permanent, so be very careful when deleting!!
#deleter(to_delete) # *UPDATE* uncomment to automate deletion of old studies

In [ ]:
browser.quit() #this line is redundant if deleter() is used

In [22]:
# testing for encoding issues

import pkgutil
import encodings
import os

def all_encodings():
    modnames = set([modname for importer, modname, ispkg in pkgutil.walk_packages(
        path=[os.path.dirname(encodings.__file__)], prefix='')])
    aliases = set(encodings.aliases.aliases.values())
    return modnames.union(aliases)

text = b'\xff'
for enc in all_encodings():
    try:
        msg = text.decode(enc)
    except Exception:
        continue
    print('Decoding {t} with {enc} is {m}'.format(t=text, enc=enc, m=msg))

Decoding b'\xff' with cp1252 is ÿ
Decoding b'\xff' with iso8859_5 is џ
Decoding b'\xff' with cp856 is  
Decoding b'\xff' with cp852 is  
Decoding b'\xff' with iso8859_4 is ˙
Decoding b'\xff' with cp1140 is 
Decoding b'\xff' with koi8_u is Ъ
Decoding b'\xff' with cp500 is 
Decoding b'\xff' with mac_croatian is ˇ
Decoding b'\xff' with cp932 is 
Decoding b'\xff' with mac_greek is ­
Decoding b'\xff' with cp1258 is ÿ
Decoding b'\xff' with cp1251 is я
Decoding b'\xff' with cp273 is 
Decoding b'\xff' with cp866 is  
Decoding b'\xff' with mac_romanian is ˇ
Decoding b'\xff' with mac_turkish is ˇ
Decoding b'\xff' with iso8859_1 is ÿ
Decoding b'\xff' with mac_centeuro is ˇ
Decoding b'\xff' with cp1257 is ˙
Decoding b'\xff' with unicode_escape is ÿ
Decoding b'\xff' with ptcp154 is я
Decoding b'\xff' with iso8859_2 is ˙
Decoding b'\xff' with koi8_r is Ъ
Decoding b'\xff' with cp869 is  
Decoding b'\xff' with cp1256 is ے
Decoding b'\xff' with iso8859_16 is ÿ
Decoding b'\xff' with cp862 is  
Decod

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: unicode_internal codec has been deprecated
  app.launch_new_instance()


In [25]:
# figuring out csv encoding
with open('roster_download_1850.csv') as f:
    print(f)

<_io.TextIOWrapper name='roster_download_1850.csv' mode='r' encoding='cp1252'>
